In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re

In [2]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [1]:
import json 
# V3=V2_3+new categories V3_1=V3+add in words 
# V3_2= + keep correct ratio 80% 
# V3_3=+~70%
# V3_4=+~60%
# V3_5=+~50%
with open ('2020_june_lexiconV3_2.json') as f1:
     dic = json.load(f1)


In [2]:
print(sorted(dic))

['covid', 'economic', 'education', 'environmental', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social cultural', 'social programs']


In [3]:
#stopword = stopwords.words('english')
# stopword = ['and','or']

In [4]:
df1 = pd.read_excel(r'C:\Users\wangtao\Documents\Illumination\data_set(2020june)\2020_TW_Pres_1000_Posts_Topic_GOLD_v2.xlsx')
# df1 = pd.read_excel('Facebook Ads Analysis.xlsx') # this data is coming from the facebook ad 2016 (the )

NameError: name 'pd' is not defined

In [307]:
df1.columns

Index(['candidate_name', 'created_at', 'favorite_count', 'reply_to',
       'retweet_count', 'retweet_status', 'original_text', 'FULL text',
       'tw_account_handle', 'text', 'GOLD'],
      dtype='object')

In [308]:
df1 = df1[['text','GOLD']]

In [309]:
df1.rename(columns={'GOLD':'Gold Label'},inplace=True)

In [310]:
# df1['Gold Label'].isnull().sum()
# three null?
# 974 in totall

In [311]:
# l = ['pre-k','12-k','anti-funges','x and y','this or that']
# l1 = []
# for phrase in l:
#     phrase = ' '.join([w.lower() for w in phrase.split() if w.lower() not in stopword])
#     phrase = re.sub("[^\w\s]",' ',phrase)
#     l1.append(phrase)            
    
# print(l1)

In [312]:
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s\-\.]",' ',x)
    #x = re.sub('[^a-zA-Z0-9]',x)
    x = re.sub("\s+",' ',x)
    ls=[w.lower() for w in x.split()]
    
    # ls=[w.lower() for w in x.split() if w.lower() not in stopword]
    
    return ' '.join(ls)

In [313]:
# s = 'pre-k,12-k,anti-funges,x and y,this or that.,u.s.'
# filter_text(s)

In [314]:
# filter the stop words in the 

# def filter_lexicon(lexicon):
#     topics=lexicon.keys()
#     for t in topics:
#         ls=[]
#         for phrase in lexicon[t]:
#             phrase = ' '.join([w.lower() for w in phrase.split()])
#             # x = re.sub("[^\w\s\-\.]",' ',x)
#             #phrase = re.sub("[^\w\s]",' ',phrase)
#             #phrase = re.sub("\s\s",' ',phrase)
#             ls.append(phrase)            
#         lexicon[t]=ls
#     return lexicon

In [315]:
# dic1 = filter_lexicon(dic)
dic1 = dic

In [316]:
# lower case the lexicon
# with open ('2020_june_lexiconV3_1.json','w') as f2:
#     json.dump(dic1,f2,sort_keys=True,indent=6)

In [317]:
df1['text'] = df1['text'].apply(lambda x: filter_text(x))

In [318]:
df1.loc[0:10,'text']

0     our children s future and the success of ameri...
1     corporate perjury seems like a good phrase to ...
2     ....are getting caught doing very evil things ...
3     when people in las vegas hear about our messag...
4     the so-called whistleblower s account of my pe...
5     newly revealed docs support flynn s innocence ...
6     it s not just facebook. giant corporations reg...
7     even more disturbing damning evidence that don...
8     twitter s new ad policy will allow fossil fuel...
9     here are just a few of trump s unqualified amb...
10    rokhanna proposed 2 amendments to reauthorizat...
Name: text, dtype: object

In [319]:
df1.dropna(axis=0,how='any',inplace=True) 

In [320]:
df1['Gold Label']=df1['Gold Label'].apply(lambda x:','.join([m.strip() for m in sorted(x.lower().split(','))]))

In [321]:
def find_words(x,lexicon):
    topics=lexicon.keys() # for version3 add in health and education and covid

    doc = nlp(x)
    words= []
    for t in topics:
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", None, *patterns)
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    words = list(set(words))
            
    return ','.join(words)
        

In [322]:
# find the words in each message
def find_topic(x,lexicon):
    topics=lexicon.keys()
    
    if x=='':
        return ''
    
    if x != '': 
        words = x.split(',')
        labels = []
        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                #print(t)
                #print(set(words)&set(terms))
                labels.append(t)
                
                #l = sorted(labels)
        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [ ]:
# get the new labels from classifer2 which depending on the updated lexicon
df1['words'] = df1['text'].astype(str).apply(lambda x: find_words(x,dic1))
#df1.rename(columns={'m_label':'words'},inplace=True)

In [ ]:
df1.tail(5)

In [ ]:
df1['m_label'] = df1['words'].apply(lambda x: find_topic(x,dic1))

In [ ]:
df1['m_label'] = df1['m_label'].apply(lambda x: 'no topic' if x=='' else x)

In [ ]:
df1.head(5)

In [257]:
# df1[['Gold Label','words','m_label']].to_csv('2020june20_tw_LabelResult.csv',index=False)

### Make the matrix and calculate precision, recall F1

In [258]:
# df1 = pd.read_csv('2020june20_tw_LabelResult.csv')

In [300]:
df2 = df1[['Gold Label', 'm_label']]

In [301]:
# pred_label = ['covid_pred','economic_pred', 'environmental_pred', 'foreign policy_pred', 'governance_pred', 
#               'immigration_pred', 'military_pred', 'safety_pred','social programs_pred', 
#               'social cultural_pred','health_pred','education_pred']

l_topic = dic1.keys()

pred_label = [x+'_pred' for x in l_topic]

pred_label
    

['covid_pred',
 'economic_pred',
 'education_pred',
 'environmental_pred',
 'foreign policy_pred',
 'governance_pred',
 'health_pred',
 'immigration_pred',
 'military_pred',
 'safety_pred',
 'social cultural_pred',
 'social programs_pred']

In [302]:
n=len(df1)
#n=1000
data0 = {x:np.zeros(n,dtype=int) for x in l_topic}
data1 = {x:np.zeros(n,dtype=int) for x in pred_label}

table0= pd.DataFrame(data0)

tabel1 = pd.DataFrame(data1)

c_df = pd.concat([df2,table0,tabel1],axis=1)

c_df.tail(10)

,Gold Label,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,...,education_pred,environmental_pred,foreign policy_pred,governance_pred,health_pred,immigration_pred,military_pred,safety_pred,social cultural_pred,social programs_pred
990,economic,economic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
991,no topic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
992,no topic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
993,no topic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
994,social cultural,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995,no topic,covid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,economic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,no topic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,no topic,no topic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,no topic,no topic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
 # in Ben and Juliar's annotation, they made some spelling mistakes of the labels like,
# typing the "social programs" as "social program", type "environment" as "environmental" etc.
# so I made this excel and correct the spelling manually.
# next step is labeling the matrix, so you may got key error because of the spelling mistakes

#c_df.to_excel('calculate_f1_june20th.xlsx',index=False)

In [263]:
# c_df = pd.read_excel('calculate_f1_june20th.xlsx')

In [264]:
for index, row in c_df.iterrows():
    #print(index)
    #print(row)
    
    if len(row['Gold Label'].split(',')) ==1 and row['Gold Label']!= 'no topic':
        c_df[row['Gold Label'].strip()][index]=1
        
    if len(row['Gold Label'].split(','))>1:
        for w in row['Gold Label'].split(','):
            #print(index)
            c_df[w.strip()][index]=1
    
    if len(row['m_label'].split(',')) ==1 and row['m_label']!= 'no topic':
        c_df[row['m_label']+'_pred'][index]=1
        
    if len(row['m_label'].split(','))>1:
        for w in row['m_label'].split(','):
            c_df[w+'_pred'][index]=1
    else: 
        pass

C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wangtao\Anaconda3\lib\site-p

In [265]:
c_df.columns

Index(['Gold Label', 'm_label', 'economic', 'environmental', 'foreign policy',
       'governance', 'immigration', 'military', 'safety', 'social cultural',
       'social programs', 'covid', 'health', 'education', 'economic_pred',
       'environmental_pred', 'foreign policy_pred', 'governance_pred',
       'immigration_pred', 'military_pred', 'safety_pred',
       'social cultural_pred', 'social programs_pred', 'covid_pred',
       'health_pred', 'education_pred'],
      dtype='object')

In [266]:
C_df = c_df.drop(columns=['Gold Label', 'm_label'])

In [267]:
C_df.shape

(999, 24)

In [157]:
# x = C_df.columns.get_loc('governance') # get the column index
# x

In [268]:
def confusion_matrix1(vectorg, vectpre):
    tp = 0 
    tn = 0 
    fp = 0
    fn = 0
    for i in range(len(vectorg)):
        if vectorg[i] == 1 and vectpre[i] == 1:
            tp = tp + 1
        elif vectorg[i] == 1 and vectpre[i] == 0:
            fn = fn + 1
        elif vectorg[i] == 0 and vectpre[i] == 0:
            tn = tn + 1
        elif vectorg[i] == 0 and vectpre[i] == 1:
            fp = fp + 1
    print('tp =', tp ,'fp =', fp ,'tn =',tn,'fn =', fn)
    print("the number of true label："+ str(tp+fn))
    #
    return (tp, fp, tn, fn)
        
def precision_recall(vectorg, vectpre):
    #print(vectorg[0],vectpre[0])
    tp, fp, tn, fn = confusion_matrix1(vectorg, vectpre)
    precision = tp/(tp + fp)
    print ('precision = ' + str(precision))
    recall = tp/ (tp + fn)
    print ('recall = ' + str(recall))
    if precision!=0 and recall!=0:
        f1 = 2/((1/precision)+(1/recall))
        print ('f1 =' + str(f1))
    if precision==0 or recall==0:
        print ('ZeroDivisionError')
        f1 = 'ZeroDivisionError'
    
    return [precision,recall,f1]
    

In [269]:
categories = list(C_df.columns.values)
categories = categories[0:12]        
print (categories)


['economic', 'environmental', 'foreign policy', 'governance', 'immigration', 'military', 'safety', 'social cultural', 'social programs', 'covid', 'health', 'education']


In [270]:
result = {}
for category in categories:
    x = C_df.columns.get_loc(category)
    print(category.strip())
    r_dic = {category.strip():precision_recall(C_df.iloc[:,x],C_df.iloc[:,x+12])}
    result.update(r_dic)
    print('\n')

economic
tp = 96 fp = 21 tn = 848 fn = 34
the number of true label：130
precision = 0.8205128205128205
recall = 0.7384615384615385
f1 =0.7773279352226721


environmental
tp = 43 fp = 18 tn = 931 fn = 7
the number of true label：50
precision = 0.7049180327868853
recall = 0.86
f1 =0.7747747747747747


foreign policy
tp = 33 fp = 17 tn = 939 fn = 10
the number of true label：43
precision = 0.66
recall = 0.7674418604651163
f1 =0.7096774193548386


governance
tp = 72 fp = 132 tn = 757 fn = 38
the number of true label：110
precision = 0.35294117647058826
recall = 0.6545454545454545
f1 =0.45859872611464975


immigration
tp = 26 fp = 1 tn = 968 fn = 4
the number of true label：30
precision = 0.9629629629629629
recall = 0.8666666666666667
f1 =0.9122807017543859


military
tp = 23 fp = 5 tn = 967 fn = 4
the number of true label：27
precision = 0.8214285714285714
recall = 0.8518518518518519
f1 =0.8363636363636364


safety
tp = 47 fp = 10 tn = 918 fn = 24
the number of true label：71
precision = 0.824561

In [271]:
re_df = pd.DataFrame(result)
re_df = re_df.T
re_df.rename(columns={0:'Precision',1:'Recall',2:'F1_Score'},inplace=True)
re_df

,Precision,Recall,F1_Score
economic,0.820513,0.738462,0.777328
environmental,0.704918,0.860000,0.774775
foreign policy,0.660000,0.767442,0.709677
governance,0.352941,0.654545,0.458599
immigration,0.962963,0.866667,0.912281
military,0.821429,0.851852,0.836364
safety,0.824561,0.661972,0.734375
social cultural,0.648649,0.615385,0.631579
social programs,0.885246,0.720000,0.794118
covid,0.205128,0.941176,0.336842


In [272]:
re_df.to_csv('2020_Twitter_CNN_june20_1.csv')